In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
import sys

In [ ]:
# Criar um modelo de Rede Neural
class Net(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size) #full connected
    self.relu = torch.nn.ReLU() #(0, infinito)
    self.fc2 = torch.nn.Linear(self.hidden_size, self.output_size)
    self.sigmoid = torch.nn.Sigmoid() #(0, 1)
  def forward(self, x):
    hidden = self.fc1(x)
    relu = self.relu(hidden)
    output = self.fc2(relu)
    output = self.sigmoid(output)
    return output

In [ ]:
# Abrir o arquivo CSV
df = pd.read_csv('Dry_Bean.csv')

FileNotFoundError: ignored

In [ ]:
# Conferir o valor máximo de cada coluna
for col in df:
  print(col, df[col].max())

In [ ]:
# Ajustar os valores de cada coluna para ficar entre 0 e 1
for column in df:
  try:
    if df[column].max() < 1:
      df[column] = df[column] / 1
    elif df[column].max() < 10:
      df[column] /= 10
    elif df[column].max() < 100:
      df[column] /= 100
    elif df[column].max() < 1000:
      df[column] /= 1000
    elif df[column].max() < 10000:
      df[column] /= 10000
    elif df[column].max() < 100000:
      df[column] /= 100000
    elif df[column].max() < 1000000:
      df[column] /= 1000000
    else:
      print('Erro')
  except:
    print('Exception')


In [ ]:
df['ShapeFactor1'] *= 10
df['ShapeFactor2'] *= 100

In [ ]:
# Conferir novamente o valor máximo de cada coluna
for col in df:
  print(col, df[col].max())

In [ ]:
# Cria um df separado com 7 colunas
df2 = pd.DataFrame(columns=list(df['Class'].unique()))
for i in range(df.shape[0]):
  if(df['Class'][i] == "SEKER"):
    df2.loc[i] = [1,0,0,0,0,0,0]
  elif(df['Class'][i] == "BARBUNYA"):
    df2.loc[i] = [0,1,0,0,0,0,0]
  elif(df['Class'][i] == "BOMBAY"):
    df2.loc[i] = [0,0,1,0,0,0,0]
  elif(df['Class'][i] == "CALI"):
    df2.loc[i] = [0,0,0,1,0,0,0]
  elif(df['Class'][i] == "HOROZ"):
    df2.loc[i] = [0,0,0,0,1,0,0]
  elif(df['Class'][i] == "SIRA"):
    df2.loc[i] = [0,0,0,0,0,1,0]
  elif(df['Class'][i] == "DERMASON"):
    df2.loc[i] = [0,0,0,0,0,0,1]
  else:
    df2.loc[i] = [9,9,9,9,9,9,9]

In [ ]:
#Remove column Class
df.drop('Class', axis=1, inplace=True)

In [ ]:
#Embaralha entrada e saída
df, df2 = shuffle(df, df2)

In [ ]:
#Separa em treinamento e teste
df2 = df2.astype('float64')
df_treino = df.iloc[:-20]
df_teste = df.iloc[-20:]
df2_treino = df2.iloc[:-20]
df2_teste = df2.iloc[-20:]

In [ ]:
#Converte pra tensor
entrada_treino = torch.FloatTensor(df_treino.values)
entrada_teste = torch.FloatTensor(df_teste.values)
saida_treino = torch.FloatTensor(df2_treino.values)
saida_teste = torch.FloatTensor(df2_teste.values)

In [ ]:
#Config do modelo
print(entrada_treino.size())
input_size = entrada_treino.size()[1]
hidden_size = 50
output_size = saida_treino.size()[1]
modelo = Net(input_size, hidden_size, output_size)
print(modelo)

# Configurações do modelo
criterion = torch.nn.MSELoss() # Mean Square Error
optimizer = torch.optim.SGD(modelo.parameters(), lr = 0.8, momentum = 0.19) 

In [ ]:
#Treinar o modelo

epochs = 5000 # Quantidade de épocas de treinamento

errors = [] # Criando um array vazio para guardar os erros de cada epoca

for epoch in range(0, epochs+1):
  optimizer.zero_grad()
  # Forward pass
  y_pred = modelo(entrada_treino)
  # Compute Loss
  loss = criterion(y_pred.squeeze(), saida_treino)
  errors.append(loss.item())
  if epoch % 1000 == 0:
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
  # Backward pass
  loss.backward()
  optimizer.step()

In [ ]:
# Config de prints
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Rodar a rede com os dados de teste, os dados que a rede nunca viu
y_pred = modelo(entrada_teste)
print(torch.round(y_pred, decimals=2)) # valor previsto pela rede
print(saida_teste) # valor real

In [ ]:
# Quantidade de erros e acertos
print(y_pred.argmax(1))
print(saida_teste.argmax(1))
erros = torch.count_nonzero(torch.sub(y_pred.argmax(1), saida_teste.argmax(1)))
acertos = y_pred.shape[0] - erros
print(f' Acertos: {acertos} / Erros: {erros}')

In [ ]:
# Gráfico evolução do erro ao longo das épocas
errors = np.array(errors)
plt.figure(figsize=(12, 8))
graf02 = plt.subplot(1, 1, 1) # nrows, ncols, index
graf02.set_title('Errors')
plt.plot(errors, '-')
plt.xlabel('Epochs')
plt.show()